# Regresion Logistica Polinomica

#### Librerias usadas

In [107]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import warnings
from sklearn.exceptions import ConvergenceWarning

#### Task 1.1

#### Leer el archivo CSV proporcionado y almacenarlo en un np.array para ser trabajado en el notebook.

In [108]:
data = pd.read_csv('framingham.csv')
warnings.filterwarnings("ignore", category=ConvergenceWarning)

#### Crear un objeto SimpleImputer para reemplazar NaN con la media de la columna en columnas numéricas

In [109]:
num_imputer = SimpleImputer(strategy='mean')

#### Aplicar la imputación a las columnas numéricas

In [110]:
data[['cigsPerDay', 'BPMeds', 'totChol', 'sysBP', 'diaBP', 'BMI', 'heartRate', 'glucose']] = num_imputer.fit_transform(data[['cigsPerDay', 'BPMeds', 'totChol', 'sysBP', 'diaBP', 'BMI', 'heartRate', 'glucose']])

#### Crear un objeto SimpleImputer para reemplazar NaN con la moda en columnas categóricas

In [111]:
cat_imputer = SimpleImputer(strategy='most_frequent')


#### Aplicar la imputación a la columna 'education'

In [112]:
data['education'] = cat_imputer.fit_transform(data['education'].values.reshape(-1, 1))

#### Convertir el DataFrame en un np.array

In [113]:
data_array = data.to_numpy()

### Task.1.2

#### Ajustar un modelo logístico polinomial en base al juego de datos cargado de forma matricial que relaciona las variables independientes que usted considere apropiadas (puede no utilizar todas las componentes de X), con la variable dependiente de salida (sufre o no sufre un paro cardíaco)..


#### Separar las características y la variable objetivo

In [114]:
X = data_array[:, :-1]
y = data_array[:, -1]

#### Escalar los datos

In [115]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

#### Separar los datos en conjuntos de entrenamiento y prueba

In [116]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

#### Aplicar transformación polinomial

In [117]:
poly = PolynomialFeatures(degree=2)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

### Task 1.4

#### Usando cross-validation determine el grado del polinomio que mejor describe la nube de puntos (encuentre el mejor balance entre apego a los datos de entrenamiento y generalización para datos previamente no observados).

In [119]:
max_degree = 5
best_degree = 1
best_score = 0

for degree in range(1, max_degree + 1):
    poly = PolynomialFeatures(degree)
    X_poly = poly.fit_transform(X)
    
    logistic_model = LogisticRegression()
    scores = cross_val_score(logistic_model, X_poly, y, cv=5)
    
    if np.mean(scores) > best_score:
        best_score = np.mean(scores)
        best_degree = degree

print(f"El mejor grado del polinomio es: {best_degree}")


El mejor grado del polinomio es: 1


## Task 1.3

### Probando el modelo y calculando el ¨Accuaracy¨ de este

In [120]:
# Crear el modelo logístico polinomial con grado 1 porque ese encontramos que es el mejor
poly = PolynomialFeatures(degree=1)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)
logistic_model = LogisticRegression()
logistic_model.fit(X_train_poly, y_train)

# Calcular la precisión del modelo en el conjunto de prueba
accuracy = logistic_model.score(X_test_poly, y_test)
print("Precisión del modelo en el conjunto de prueba:", accuracy)


Precisión del modelo en el conjunto de prueba: 0.8584905660377359


### Task 1.5

#### Haga un análisis sobre sus hallazgos.

##### Los hallazgos fueron que se realizo un modelo de regresion logística polinomial para predecir si un paciente sufrirá o no un paro cardíaco, para eso ptimero se realizo una exploración a los datos evaluando los diferentes grados de polinomios y evaluar el rendimiento del modelo utilizando validación cruzada para seleccionar el grado óptimo que mejor describe los datos. Y el resultado dado por el programa es que el mejor grado del polinomio es 1 para este conjunto de datos brindado. El grado 1 es el que mejor describe la nube de puntos. Luego investigue cuales eran las mejores metricas de desempeño para aplicarle al modelo y elegí calcular el de presición(accuaracy) el cual dio un resultado bastante acetado de 0.86(redondeando).